In [15]:
!python -V

Python 3.12.7


In [16]:
import pandas as pd
import pickle

In [17]:
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [19]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [20]:
#Q1
df.shape

(3066766, 19)

In [21]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [22]:
list(set(b) - set(a))

['lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'trip_type',
 'duration',
 'ehail_fee']

In [23]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)


In [24]:
#Q2
df.duration.std()

42.594351241920904

In [25]:
#Q3
df[(df.duration>=1) & (df.duration<=60)]["VendorID"].count()/df["VendorID"].count()

0.9812202822125979

In [26]:
df = df[(df.duration>=1) & (df.duration<=60)]
#df = df.drop('airport_fee',axis=1)

In [27]:
#Q4
categorical = [ 'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

In [28]:
train_dicts = df[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [29]:
X_train.shape

(3009173, 516)

In [30]:
#Q5
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.6584030070241

In [31]:
#Q6

In [32]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [33]:
df_val = read_dataframe("./data/yellow_tripdata_2023-02.parquet")

In [34]:
df_val = df_val.rename(columns={'Airport_fee': 'airport_fee'})

In [35]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
dv_val = DictVectorizer()
X_val = dv_val.fit_transform(val_dicts)

In [ ]:
y_pred_val = lr.predict(X_val)